## Building a MongoDB Data Model

- The relationship between climate data and fire data is one-to-many relationship. It is guaranteed that one climate data will be recorded on a daily basis regardless if there are any fire data on that particular day, and a fire data will only be connected to a single climate data by referencing on the date attribute. Therefore, the query operation will be more efficient to retrieve both the climate data and fire data.

Example of the data model:
~~~

[
 {
   "station":"948701",
   "date":"6/3/21",
   "air_temperature_celcius":20,
   "relative_humidity":52.9,
   "windspeed_knots":17.0,
   "max_wind_speed":21.0,
   "GHI_w/m2":167,
   "precipitation_flag":"I",
   "precipitation_value":0.0,
   "cause_of_fire": "other",
   "hotspot":[
      {
         "latitude":-34.3795,
         "longitude":141.6331,
         "datetime":"2021-03-06T05:06:30",
         "confidence":87,
         "surface_temperature_celcius":62
      },
      {
         "latitude":-34.3735,
         "longitude":141.6604,
         "datetime":"2021-03-06T05:06:20",
         "confidence":85,
         "surface_temperature_celcius":59
      }
   ]
 },
...
]

~~~

## Querying MongoDB using PyMongo

In [1]:
# Import necessary libraries
import pymongo
from pymongo import MongoClient
from pprint import pprint
from datetime import datetime
import csv

In [2]:
client = MongoClient()
db = client.fit3182_assignment_db
collection = db.data

In [3]:
# Function to parse csv to dictionary
def convert_csv_to_dict(file_path, int_list, float_list):
    reader = csv.DictReader(open(file_path,'r'))
    content = list(reader)
    
    # Iterate over each row
    for row in content:
        # Convert string to int
        for col in int_list:
            row[col] = int(row[col])
            
        # Convert string to float
        for col in float_list:
            # Separate precipitation flag from value
            if col == 'precipitation ':
                item = row[col].strip()
                flag = item[-1]
                value = float(item[:len(item)-1])
                row['precipitation_flag'] = flag
                row['precipitation_value'] = value
                
                # Remove key
                row.pop('precipitation ')
            else:
                row[col] = float(row[col])
                
    return content

In [4]:
# Separate columns into int list and float list
cl_int_list = ['GHI_w/m2','air_temperature_celcius']
cl_float_list = ['max_wind_speed','windspeed_knots','relative_humidity','precipitation ']

# Obtain dictionary data from climate_historic.csv
cl_dict = convert_csv_to_dict("climate_historic.csv", cl_int_list, cl_float_list)

In [5]:
# Separate columns into int list and float list
fr_int_list = ['confidence','surface_temperature_celcius']
fr_float_list = ['latitude','longitude']

# Obtain dictionary data from hotspot_historic.csv
fr_dict = convert_csv_to_dict("hotspot_historic.csv", fr_int_list, fr_float_list)

In [6]:
# Group data with the same date 
fr_join_dict = {}

for item in fr_dict:
    key = datetime.strptime(item['date'],'%d/%m/%Y').date()
    item.pop('date')
    try:
        fr_join_dict[key].append(item)
    except KeyError:
        fr_join_dict[key] = [item]

In [7]:
# Embed hotspot data into climate data
for item in cl_dict:
    key = datetime.strptime(item['date'],'%d/%m/%y').date()
    
    try:
        item['hotspot'] = fr_join_dict[key]
        
        # For Part B: Label cause of fire
        if item['GHI_w/m2'] > 180 and item['air_temperature_celcius'] > 20:
            item['cause_of_fire'] = 'natural'
        else:
            item['cause_of_fire'] = 'other'
    except KeyError:
        pass

In [8]:
collection.insert_many(cl_dict)

### Q2

In [9]:
# Function to print document in cursor
def print_document(cursor):
    for document in cursor:
        pprint(document)

In [10]:
# a. Find climate data on 12th December 2021
cursor = collection.find({'date':'12/12/21'}, {'_id':0})
print_document(cursor)

{'GHI_w/m2': 156,
 'air_temperature_celcius': 19,
 'cause_of_fire': 'other',
 'date': '12/12/21',
 'hotspot': [{'confidence': 53,
              'datetime': '2021-12-12T00:45:38',
              'latitude': -37.903,
              'longitude': 145.25,
              'surface_temperature_celcius': 44}],
 'max_wind_speed': 12.0,
 'precipitation_flag': 'I',
 'precipitation_value': 0.0,
 'relative_humidity': 55.3,
 'station': '948702',
 'windspeed_knots': 6.2}


In [11]:
# b. Find the latitude, longitude, surface temperature and confidence when
# the surface temperature was between 65 and 100.
cursor = collection.aggregate([
    {
        "$unwind": "$hotspot"
    },
    {
        "$match": {"hotspot.surface_temperature_celcius": {"$gte":65,"$lte":100}}
    },
    {
        "$project":{"hotspot.latitude":1,
                    "hotspot.longitude":1,
                    "hotspot.surface_temperature_celcius":1,
                    "hotspot.confidence":1,
                    "_id":0}
    }
])
print_document(cursor)

{'hotspot': {'confidence': 94,
             'latitude': -37.2284,
             'longitude': 147.9187,
             'surface_temperature_celcius': 73}}
{'hotspot': {'confidence': 97,
             'latitude': -37.6572,
             'longitude': 142.0703,
             'surface_temperature_celcius': 80}}
{'hotspot': {'confidence': 84,
             'latitude': -37.0193,
             'longitude': 148.1459,
             'surface_temperature_celcius': 71}}
{'hotspot': {'confidence': 100,
             'latitude': -37.4229,
             'longitude': 147.027,
             'surface_temperature_celcius': 99}}
{'hotspot': {'confidence': 80,
             'latitude': -37.0055,
             'longitude': 148.1582,
             'surface_temperature_celcius': 68}}
{'hotspot': {'confidence': 85,
             'latitude': -37.4128,
             'longitude': 147.0242,
             'surface_temperature_celcius': 98}}
{'hotspot': {'confidence': 90,
             'latitude': -34.357,
             'longitude': 141

In [12]:
# c. Find date, surface temperature, air temperature, relative humidity and max wind speed
# on 15th and 16th of December 2021
cursor = collection.find(
    {
        "date" : {"$in":["15/12/21","16/12/21"]}
    },
    {
        "date":1, 
        "hotspot.surface_temperature_celcius": 1, 
        "air_temperature_celcius": 1,
        "relative_humidity":1,
        "max_wind_speed":1,
        "_id":0
    })
print_document(cursor)

{'air_temperature_celcius': 18,
 'date': '15/12/21',
 'hotspot': [{'surface_temperature_celcius': 42},
             {'surface_temperature_celcius': 36},
             {'surface_temperature_celcius': 38},
             {'surface_temperature_celcius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'air_temperature_celcius': 18,
 'date': '16/12/21',
 'hotspot': [{'surface_temperature_celcius': 43},
             {'surface_temperature_celcius': 33},
             {'surface_temperature_celcius': 54},
             {'surface_temperature_celcius': 73},
             {'surface_temperature_celcius': 55},
             {'surface_temperature_celcius': 75},
             {'surface_temperature_celcius': 55},
             {'surface_temperature_celcius': 66},
             {'surface_temperature_celcius': 56},
             {'surface_temperature_celcius': 60},
             {'surface_temperature_celcius': 73},
             {'surface_temperature_celcius': 48},
             {'surface_temperature_celci

In [11]:
# d. Find datetime, air temperature, surface temperature and confidence 
# when the confidence is between 80 and 100
cursor = collection.find(
    {
        "hotspot.confidence": {"$gte":80, "$lte": 100}
    }, 
    {
        "hotspot.datetime": 1,
        "air_temperature_celcius":1,
        "hotspot.surface_temperature_celcius":1,
        "_id":0
    })
print_document(cursor)

{'air_temperature_celcius': 20,
 'hotspot': [{'datetime': '2021-03-06T05:06:30',
              'surface_temperature_celcius': 62},
             {'datetime': '2021-03-06T05:06:20',
              'surface_temperature_celcius': 59}]}
{'air_temperature_celcius': 19,
 'hotspot': [{'datetime': '2021-03-07T04:16:10',
              'surface_temperature_celcius': 64}]}
{'air_temperature_celcius': 23,
 'hotspot': [{'datetime': '2021-03-09T13:23:40',
              'surface_temperature_celcius': 41},
             {'datetime': '2021-03-09T03:57:00',
              'surface_temperature_celcius': 44},
             {'datetime': '2021-03-09T03:56:50',
              'surface_temperature_celcius': 55}]}
{'air_temperature_celcius': 19,
 'hotspot': [{'datetime': '2021-03-10T04:48:40',
              'surface_temperature_celcius': 105},
             {'datetime': '2021-03-10T04:46:20',
              'surface_temperature_celcius': 109},
             {'datetime': '2021-03-10T04:45:30',
              'surface_tem

In [12]:
# e. Find the top 10 records with the highest surface temperature
cursor = collection.aggregate([
    {
        "$unwind":"$hotspot"
    },
    {
        "$sort": {
            "hotspot.surface_temperature_celcius":-1
        }
    },
    {
        "$limit":10
    }
])
print_document(cursor)

{'GHI_w/m2': 122,
 '_id': ObjectId('628dfe1c2c4f1386af17d743'),
 'air_temperature_celcius': 15,
 'cause_of_fire': 'other',
 'date': '18/4/21',
 'hotspot': {'confidence': 100,
             'datetime': '2021-04-18T04:52:00',
             'latitude': -38.1665,
             'longitude': 143.062,
             'surface_temperature_celcius': 124},
 'max_wind_speed': 9.9,
 'precipitation_flag': 'I',
 'precipitation_value': 0.0,
 'relative_humidity': 56.1,
 'station': '948701',
 'windspeed_knots': 5.1}
{'GHI_w/m2': 140,
 '_id': ObjectId('628dfe1c2c4f1386af17d735'),
 'air_temperature_celcius': 16,
 'cause_of_fire': 'other',
 'date': '4/4/21',
 'hotspot': {'confidence': 100,
             'datetime': '2021-04-04T04:32:50',
             'latitude': -36.343,
             'longitude': 142.1986,
             'surface_temperature_celcius': 123},
 'max_wind_speed': 12.0,
 'precipitation_flag': 'I',
 'precipitation_value': 0.0,
 'relative_humidity': 47.5,
 'station': '948701',
 'windspeed_knots': 5.4}
{'

In [13]:
# f. Find the number of fires each day.
cursor = collection.aggregate([
    {
        "$project":{
            "_id":0,
            "date":1,
            "number_of_fire":{
                "$size":{
                    "$ifNull":["$hotspot",[]]
                }
            }
        }
    }
])
print_document(cursor)

{'date': '31/12/20', 'number_of_fire': 0}
{'date': '2/1/21', 'number_of_fire': 0}
{'date': '3/1/21', 'number_of_fire': 0}
{'date': '4/1/21', 'number_of_fire': 0}
{'date': '5/1/21', 'number_of_fire': 0}
{'date': '6/1/21', 'number_of_fire': 0}
{'date': '7/1/21', 'number_of_fire': 0}
{'date': '8/1/21', 'number_of_fire': 0}
{'date': '9/1/21', 'number_of_fire': 0}
{'date': '10/1/21', 'number_of_fire': 0}
{'date': '11/1/21', 'number_of_fire': 0}
{'date': '12/1/21', 'number_of_fire': 0}
{'date': '13/1/21', 'number_of_fire': 0}
{'date': '14/1/21', 'number_of_fire': 0}
{'date': '15/1/21', 'number_of_fire': 0}
{'date': '16/1/21', 'number_of_fire': 0}
{'date': '17/1/21', 'number_of_fire': 0}
{'date': '18/1/21', 'number_of_fire': 0}
{'date': '19/1/21', 'number_of_fire': 0}
{'date': '20/1/21', 'number_of_fire': 0}
{'date': '21/1/21', 'number_of_fire': 0}
{'date': '22/1/21', 'number_of_fire': 0}
{'date': '23/1/21', 'number_of_fire': 0}
{'date': '24/1/21', 'number_of_fire': 0}
{'date': '25/1/21', 'nu

In [14]:
# g. Find the records of fires where the confidence is below 70
cursor = collection.aggregate([
    {
        "$unwind": "$hotspot"
    },
    {
        "$match": {"hotspot.confidence":{"$lte":70}}
    },
    {
        "$project":{"hotspot":1,"_id":0}
    }
])
print_document(cursor)

{'hotspot': {'confidence': 68,
             'datetime': '2021-03-08T04:51:00',
             'latitude': -37.7885,
             'longitude': 141.9352,
             'surface_temperature_celcius': 55}}
{'hotspot': {'confidence': 54,
             'datetime': '2021-03-09T03:57:00',
             'latitude': -37.7171,
             'longitude': 147.5866,
             'surface_temperature_celcius': 44}}
{'hotspot': {'confidence': 55,
             'datetime': '2021-03-10T04:43:00',
             'latitude': -36.2544,
             'longitude': 148.0353,
             'surface_temperature_celcius': 42}}
{'hotspot': {'confidence': 54,
             'datetime': '2021-03-10T04:42:30',
             'latitude': -37.2197,
             'longitude': 147.9621,
             'surface_temperature_celcius': 43}}
{'hotspot': {'confidence': 56,
             'datetime': '2021-03-13T23:58:50',
             'latitude': -37.0286,
             'longitude': 148.155,
             'surface_temperature_celcius': 42}}
{'hots

In [17]:
# h. Find the average surface temperature for each day
cursor = collection.aggregate([
    {
        "$project": {
            "_id":0,
            "date":1,
            "avg_surface_temperature":{
                "$avg": "$hotspot.surface_temperature_celcius"
            }
        }
    }
])
print_document(cursor)

{'avg_surface_temperature': None, 'date': '31/12/20'}
{'avg_surface_temperature': None, 'date': '2/1/21'}
{'avg_surface_temperature': None, 'date': '3/1/21'}
{'avg_surface_temperature': None, 'date': '4/1/21'}
{'avg_surface_temperature': None, 'date': '5/1/21'}
{'avg_surface_temperature': None, 'date': '6/1/21'}
{'avg_surface_temperature': None, 'date': '7/1/21'}
{'avg_surface_temperature': None, 'date': '8/1/21'}
{'avg_surface_temperature': None, 'date': '9/1/21'}
{'avg_surface_temperature': None, 'date': '10/1/21'}
{'avg_surface_temperature': None, 'date': '11/1/21'}
{'avg_surface_temperature': None, 'date': '12/1/21'}
{'avg_surface_temperature': None, 'date': '13/1/21'}
{'avg_surface_temperature': None, 'date': '14/1/21'}
{'avg_surface_temperature': None, 'date': '15/1/21'}
{'avg_surface_temperature': None, 'date': '16/1/21'}
{'avg_surface_temperature': None, 'date': '17/1/21'}
{'avg_surface_temperature': None, 'date': '18/1/21'}
{'avg_surface_temperature': None, 'date': '19/1/21'}


In [18]:
# i. Find the top 10 records with the lowest GHI.
cursor = collection.aggregate([
    {
        "$sort":{"GHI_w/m2":1}
    },
    {
        "$limit":10
    }
])
print_document(cursor)

{'GHI_w/m2': 47,
 '_id': ObjectId('628cf69e2c4f13d2a9759015'),
 'air_temperature_celcius': 5,
 'cause_of_fire': 'other',
 'date': '2/8/21',
 'hotspot': [{'confidence': 94,
              'datetime': '2021-08-02T03:45:40',
              'latitude': -37.4796,
              'longitude': 141.9403,
              'surface_temperature_celcius': 87},
             {'confidence': 54,
              'datetime': '2021-08-02T03:45:00',
              'latitude': -37.491,
              'longitude': 141.936,
              'surface_temperature_celcius': 40}],
 'max_wind_speed': 5.1,
 'precipitation_flag': 'I',
 'precipitation_value': 0.0,
 'relative_humidity': 38.6,
 'station': '948701',
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48,
 '_id': ObjectId('628cf69e2c4f13d2a9758ff4'),
 'air_temperature_celcius': 5,
 'cause_of_fire': 'other',
 'date': '30/6/21',
 'hotspot': [{'confidence': 78,
              'datetime': '2021-06-30T04:41:25',
              'latitude': -36.834,
              'longitude': 142.524,
    

In [19]:
# j. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35
cursor = collection.find(
    {
        "$and":[{"precipitation_flag":"G"},{"precipitation_value":{"$gte":0.20,"$lte":0.35}}]
    }, 
    {
        "_id": 0    
    }
)
print_document(cursor)

{'GHI_w/m2': 157,
 'air_temperature_celcius': 19,
 'date': '13/1/21',
 'max_wind_speed': 18.1,
 'precipitation_flag': 'G',
 'precipitation_value': 0.31,
 'relative_humidity': 54.1,
 'station': '948700',
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146,
 'air_temperature_celcius': 17,
 'cause_of_fire': 'other',
 'date': '29/3/21',
 'hotspot': [{'confidence': 69,
              'datetime': '2021-03-29T00:48:40',
              'latitude': -34.2648,
              'longitude': 141.6325,
              'surface_temperature_celcius': 51}],
 'max_wind_speed': 21.0,
 'precipitation_flag': 'G',
 'precipitation_value': 0.24,
 'relative_humidity': 49.9,
 'station': '948701',
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166,
 'air_temperature_celcius': 20,
 'cause_of_fire': 'other',
 'date': '20/4/21',
 'hotspot': [{'confidence': 84,
              'datetime': '2021-04-20T04:44:20',
              'latitude': -36.8871,
              'longitude': 145.1536,
              'surface_temperature_celcius': 58},
         